In [1]:
dataset_path = '../bin/multirc_dataset.hf'
model_name = '/tiedaar/summary-longformer-wording'

In [6]:
from transformers import pipeline
from datasets import DatasetDict, Dataset
import pandas as pd
from tqdm import tqdm
import torch

tqdm.pandas()

In [3]:
pipe = pipeline('text-classification', model='tiedaar/short-answer-classification', device=0)

tokenizer_config.json:   0%|          | 0.00/20.8k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/3.58M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/694 [00:00<?, ?B/s]

Device set to use cpu


In [8]:
ds = DatasetDict.load_from_disk(dataset_path)

FileNotFoundError: No such file: '/home/jovyan/active-projects/itell-question-generation/src/mpnet/../bin/multirc_dataset.hf/dataset_dict.json'. Expected to load a `DatasetDict` object, but provided path is not a `DatasetDict`.

In [4]:
test_df = ds['test'].to_pandas()

NameError: name 'ds' is not defined

In [7]:
df = pd.read_csv('../data/multirc.csv')
column_index = list(df.columns)
column_index[0] = 'index'
df.columns = column_index
test_df = df.merge(test_df, on='index', how='right')
test_df['labels'] = test_df['labels'].progress_apply(lambda x: 'correct_answer' if x==1 else 'incorrect_answer')

In [8]:
test_df['preds'] = test_df['text'].progress_apply(lambda x: pipe(x)[0]['label'])

  0%|          | 7/3962 [00:01<12:01,  5.48it/s]/home/jovyan/conda_envs/wes_env/lib/python3.11/site-packages/transformers/pipelines/base.py:1083: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(
100%|██████████| 3962/3962 [01:00<00:00, 65.88it/s]


In [14]:
test_df[test_df['labels']!=test_df['preds']].to_csv('../results/mpnet-misclassifications.csv')